In [4]:
import sys, os
sys.path.append("C:\\Users\\Administrator\\Desktop\\Deeptrain")
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'Deeptrain.settings')
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'rest.settings')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

import django
django.setup()

from utils.benchmark import *
from utils.cache import hash_cache_process_safe as hash_cache
from utils.cache import integer_operation
from utils.throttle import rate_throttle
from user.models import User
from blog.models import Article
def req():
    return Article.objects.filter(author=User.objects.get()).count()

In [2]:
[fps_analysis(req, count=1000) for _ in range(15)]  # 预热cache

[350.36589030132376,
 566.5262114867088,
 558.8596146200083,
 581.2723842594422,
 577.5986198128683,
 575.3486066297194,
 572.461102695272,
 549.1960308339305,
 562.9722134156997,
 587.5412187584924,
 604.876619304924,
 577.7997701091156,
 419.43723682696026,
 358.59135961763974,
 567.3856954225412]

In [3]:
fps_analysis(req, count=1000)

583.05012366328

In [4]:
fps_compare(hash_cache(req), req, count=10**4)

'function fps: 65351.076874656246, compared function fps: 566.0337763798809'

In [11]:
integer_operation("some-cache", expiration=1)

1

In [12]:
integer_operation("asdf", expiration=1)

2

In [14]:
integer_operation("asdf", expiration=1)

1

In [30]:
integer_operation("asdf", expiration=10)

5

In [2]:
[rate_throttle("somekey", 99) for _ in range(100)]

[False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 True]

In [3]:
from time import sleep

for _ in range(10):
    sleep(1)
    print([rate_throttle("val", 9, expiration=1, version="version-2") for _ in range(10)])

[False, False, False, False, False, False, False, False, False, True]
[False, False, False, False, False, False, False, False, False, True]
[False, False, False, False, False, False, False, False, False, True]
[False, False, False, False, False, False, False, False, False, True]
[False, False, False, False, False, False, False, False, False, True]
[False, False, False, False, False, False, False, False, False, True]
[False, False, False, False, False, False, False, False, False, True]
[False, False, False, False, False, False, False, False, False, True]
[False, False, False, False, False, False, False, False, False, True]
[False, False, False, False, False, False, False, False, False, True]


In [8]:
for _ in range(6):
    sleep(1)
    print([rate_throttle("val", 9, expiration=1.01, version="version-1", touch=True) for _ in range(12)])

[False, False, False, False, False, False, False, False, False, True, True, True]
[True, True, True, True, True, True, True, True, True, True, True, True]
[True, True, True, True, True, True, True, True, True, True, True, True]
[False, False, False, False, False, False, False, False, False, True, True, True]
[False, False, False, False, False, False, False, False, False, True, True, True]
[True, True, True, True, True, True, True, True, True, True, True, True]
